## pipeline

In [1]:
import networkx as nx
import numpy as np
import torch

import torch.nn as nn

from statistics import mean

from func.run_pipeline_super_vox import get_outlayer_of_a_3d_shape, get_crop_by_pixel_val
from func.ultis import load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset


Using backend: pytorch


In [2]:
# load graphs
from func.ultis import load_obj

# graphs = load_obj("graphs_dataset_train")
graphs = load_obj("graphs_dataset_train_with_augmentations")

In [3]:
import random
random.seed(0)
random.shuffle(graphs)

In [4]:
dataset = VoxelGraphDataset(graphs, with_edge_weights=True)

g = dataset[0]


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  return th.as_tensor(data, dtype=dtype)


# TODO probably should normalize features!!!!

In [5]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples)

"""
train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)
"""

'\ntrain_sampler = SubsetRandomSampler(torch.arange(num_train))\ntest_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))\n\ntrain_dataloader = GraphDataLoader(\n    dataset, sampler=train_sampler, batch_size=5, drop_last=False)\n'

In [6]:
from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random

model = GCN(3, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""
from torchmetrics import F1Score
from torchvision.ops import sigmoid_focal_loss

f1 = F1Score(num_classes=2, average='weighted')

epoch_loss = []
epoch_val_loss = []
epoch_accuracy = []

epoch_f1score = []
epoch_f1score_val = []

epoch_accuracy_val = []
# best_val_acc = 0
best_val_loss = 1000

for e in range(500):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    for graph_number in range(len(dataset)):
    #for graph_number in range(1):
    #for graph_number in rand_graph_numbers:
        # Forward
        model.train()
        sample_graph = dataset[graph_number]
        features = sample_graph.ndata['feat']
        labels = sample_graph.ndata['label']
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']

        # create class weights
        number_positives = torch.count_nonzero(labels)
        percentage_positives = number_positives / len(labels)
        percentage_negatives = 1 - percentage_positives

        weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
        # print(f"weights: {weights}")
        weights = torch.tensor([0.6, 0.4])
        # print(f"weights: {weights}")
        # print(f"weights: {weights}")

        # CELoss = nn.CrossEntropyLoss(weight=weights)
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']
        logits = model(sample_graph, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(f"num of predicted negatives: {len(pred[pred==0])}")
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        # loss = CELoss(logits[train_mask], labels[train_mask])
        loss = sigmoid_focal_loss(logits[train_mask].type(torch.FloatTensor), F.one_hot(labels[train_mask], num_classes=2).type(torch.FloatTensor), alpha=0.1, reduction="mean")



        epoch_loss.append(loss.item())

        #print(loss)
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()

        train_f1_score = f1(pred[train_mask], labels[train_mask])
        val_f1_score = f1(pred[val_mask], labels[val_mask])


        epoch_accuracy.append(train_acc.item())
        epoch_accuracy_val.append(val_acc.item())

        epoch_f1score.append(train_f1_score.item())
        epoch_f1score_val.append(val_f1_score.item())


        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            logits = model(sample_graph, features)
            # val_loss = CELoss(logits[val_mask], labels[val_mask])
            val_loss = sigmoid_focal_loss(logits[val_mask].type(torch.FloatTensor), F.one_hot(labels[val_mask], num_classes=2).type(torch.FloatTensor), alpha=0.8, reduction="mean")
            epoch_val_loss.append(val_loss.item())
        model.train()

    if e % 5 == 0:
        print('In epoch {}, loss: {:.5f}, val loss: {:.5f}, accuracy: {:.3f}, val accuracy: {:.3f}, f1score: {:.3f}, val f1score: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_val_loss), mean(epoch_accuracy), mean(epoch_accuracy_val), mean(epoch_f1score), mean(epoch_f1score_val)))

        #if mean(epoch_accuracy_val) >= best_val_acc:
        if mean(epoch_val_loss) <= best_val_loss:
            print("new best val loss")
            torch.save(model.state_dict(), "output/graph_model.pt")
            best_val_loss = mean(epoch_val_loss)
        epoch_loss = []
        epoch_val_loss = []
        epoch_accuracy = []

        epoch_accuracy_val = []
        epoch_f1score_val = []


num of predicted negatives: 1
num of predicted negatives: 0
num of predicted negatives: 1
num of predicted negatives: 1
num of predicted negatives: 0
num of predicted negatives: 1
num of predicted negatives: 0
num of predicted negatives: 2
num of predicted negatives: 0
num of predicted negatives: 1
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 2
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of predicted negatives: 0
num of pre


KeyboardInterrupt



from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random
import dgl

model = GCN(1, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""

# build one big graph
graphs_list = []
for i in range(len(dataset)):
    graphs_list.append(dataset[i])

large_g = dgl.batch(graphs_list)

epoch_loss = []
epoch_accuracy = []
for e in range(1000):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    sample_graph = large_g
    features = sample_graph.ndata['feat']
    labels = sample_graph.ndata['label']

    # create class weights
    number_positives = torch.count_nonzero(labels)
    percentage_positives = number_positives / len(labels)
    percentage_negatives = 1 - percentage_positives

    weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
    #weights = torch.tensor([0.95, 0.05])
    #print(weights)

    CELoss = nn.CrossEntropyLoss(weight=weights)
    #train_mask = sample_graph.ndata['train_mask']
    #val_mask = sample_graph.ndata['val_mask']
    logits = model(sample_graph, features)

    # Compute prediction
    pred = logits.argmax(1)

    # Compute loss
    # Note that you should only compute the losses of the nodes in the training set.
    loss = CELoss(logits, labels)
    epoch_loss.append(loss.item())
    #print(loss)
    # Compute accuracy on training/validation/test
    train_acc = (pred == labels).float().mean()
    epoch_accuracy.append(train_acc.item())
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 20 == 0:
        print('In epoch {}, loss: {:.3f}, accuracy: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_accuracy)))
        epoch_loss = []
        epoch_accuracy = []




# Train on GPU
g = g.to('cuda')
model = GCN(1, 16, dataset.num_classes).to('cuda')
train(g, model)

In [ ]:
g

In [ ]:

g.ndata['feat'].shape

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(g, g.ndata['feat']).argmax(1).numpy()

In [ ]:
np.unique(predictions)

In [ ]:
labels = g.ndata['label']
np.unique(labels)

In [ ]:
len(labels[labels==1])

In [ ]:
len(labels[labels==0])

In [ ]:
len(predictions[predictions==1])

In [ ]:
len(predictions[predictions==0])

In [ ]:
graph_model = GCN(3, num_classes=2)
checkpoint_graph = 'output/graph_model.pt'
graph_model.load_state_dict(torch.load(checkpoint_graph))
graph_model.eval()

In [ ]:
predictions = graph_model(g, g.ndata['feat']).argmax(1).numpy()


In [ ]:

len(predictions[predictions==1])

In [ ]:
len(predictions[predictions==0])